In [1]:
from rrt_star import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import roboticstoolbox as rtb
import spatialmath as sm # provides objects for representing transformations
from swift import Swift # lightweight browser-based simulator which comes with the toolbox
import spatialgeometry as sg # utility package for dealing with geometric objects

In [3]:
robot = rtb.models.wx200()
print(robot)
useful_links = lambda robot: [robot.links[i] for i in (0,1,2,3,4)]

ERobot: wx200 (by Interbotix), 8 joints (RRRRRRPP), 4 branches, dynamics, geometry, collision
┌──────┬─────────────────────┬───────┬───────────────────┬───────────────────────────────────────────┐
│ link │        link         │ joint │      parent       │            ETS: parent to link            │
├──────┼─────────────────────┼───────┼───────────────────┼───────────────────────────────────────────┤
│    0 │ /base_link          │       │ BASE              │ SE3()                                     │
│    1 │ /shoulder_link      │     0 │ /base_link        │ SE3(0, 0, 0.0716) ⊕ Rz(q0)                │
│    2 │ /upper_arm_link     │     1 │ /shoulder_link    │ SE3(0, 0, 0.03865) ⊕ Ry(q1)               │
│    3 │ /forearm_link       │     2 │ /upper_arm_link   │ SE3(0.05, 0, 0.2; 180°, -0°, 0°) ⊕ Ry(q2) │
│    4 │ /wrist_link         │     3 │ /forearm_link     │ SE3(0.2, 0, 0) ⊕ Ry(q3)                   │
│    5 │ /gripper_link       │     4 │ /wrist_link       │ SE3(0.065, 0, 0; -180°,

In [4]:
def has_pairwise_collision(links):
	n_links = len(links)
	for i1 in range(n_links):
		link1 = links[i1]
		for i2 in list(range(0,i1-1))+list(range(i1+2,n_links)):
			link2 = links[i2]
			for collider1 in link1.collision:
				if collider1.stype == 'mesh':
					continue
				for collider2 in link2.collision:
					if collider2.stype == 'mesh':
						continue
					if collider1.iscollided(collider2):
						print(collider1, '\n with \n', collider2)
						return True

	return False

has_pairwise_collision(useful_links(robot))

False

In [10]:
start_config = robot.q # Initial 6D configuration
goal_config = (1/4*robot.qlim[0]+3/4*robot.qlim[1])  # Goal 6D configuration

rrt_star = RRTStar(start=start_config, goal=goal_config, minCoords=robot.qlim[0], maxCoords=robot.qlim[1], step_size=0.1)
path = rrt_star.rrt_star()

if path:
	print("Found path:")
	for config in path:
		print(config)
else:
	print("Path not found.")

Goal reached in 342 iterations.
Found path:
[ 0.          0.06544985  0.19634954 -0.3010693   0.          0.
  0.037      -0.037     ]
[0.04942258665314923, 0.09510339894167698, 0.2218845439534891, -0.26729719508936994, 0.04942258665314923, 0.04942258665314923, 0.0368269513227428, -0.03648085396822841]
[0.7310771377616941, 0.23012873696552044, 0.28332334773633666, -0.03443462702381026, 0.33093128612640765, 0.6441826559291475, 0.03368146400371832, -0.03127827294684427]
[1.118568988169904, 0.5227812620206882, 0.5253722023018081, 0.3523165551386553, 0.8977999473663282, 1.1081089130634152, 0.03266375467767614, -0.02647523473667026]
[1.4339197783925237, 0.7507320579479693, 0.8045563097030664, 0.6677052529327987, 1.4268471692334936, 1.441870632283838, 0.03089825254924907, -0.02423945182340989]
[ 1.57079633  1.00792764  1.00792764  0.77230819  1.57079633  1.57079633
  0.0315     -0.0205    ]


In [16]:
ee_axes = sg.Axes(0.1)
goal_axes = sg.Axes(0.1)
goal_axes.T = robot.fkine(goal_config, end='/ee_arm_link')

# Make a new environment and add our robot
env = Swift()
env.launch(realtime=True, browser="notebook")
env.add(robot)
env.add(ee_axes)
env.add(goal_axes)

# Change the robot configuration to the ready position
# robot._T = np.diag(np.ones(4))
robot.qr = np.zeros(8)
robot.q = robot.qr

# Step the sim to view the robot in this configuration
env.step(0.01)

dt = 0.5
time = 0

for step_i in range(len(path)):
	robot.q = path[step_i]

	# Update the ee axes
	ee_axes.T = robot.fkine(robot.q, end='/ee_arm_link').A
	
	# Step the simulator by dt seconds
	env.step(dt)
	time += dt